# **COVID-19 Dash Webapp**

**Imports & Installs**

In [ ]:
pip install dash

In [ ]:
pip install dash_core_components

In [ ]:
pip install dash_html_components

In [ ]:
pip install dash_bootstrap_components

In [1]:
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output

**Data collection & pre-processing**

In [2]:
url = "https://covid.ourworldindata.org/data/owid-covid-data.csv"
df = pd.read_csv(url)

In [3]:
df = df.loc[~df['location'].isin(['International'])]
df.loc[df['location']=='World', 'continent'] = 'World'

In [4]:
df["scaled"] = df["total_cases"] ** 0.77

In [5]:
df

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,scaled
0,AFG,Asia,Afghanistan,2020-01-23,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498,NaN
1,AFG,Asia,Afghanistan,2020-01-24,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498,NaN
2,AFG,Asia,Afghanistan,2020-01-25,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498,NaN
3,AFG,Asia,Afghanistan,2020-01-26,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498,NaN
4,AFG,Asia,Afghanistan,2020-01-27,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62585,ZWE,Africa,Zimbabwe,2020-12-10,11081.0,74.0,93.857,305.0,1.0,3.571,...,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.535,1301.145294
62586,ZWE,Africa,Zimbabwe,2020-12-11,11162.0,81.0,87.857,306.0,1.0,3.571,...,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.535,1308.462722
62587,ZWE,Africa,Zimbabwe,2020-12-12,11219.0,57.0,86.000,307.0,1.0,2.286,...,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.535,1313.604701
62588,ZWE,Africa,Zimbabwe,2020-12-13,11246.0,27.0,75.429,307.0,0.0,2.286,...,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.535,1316.038277


**Designing Webapp Dashboard**

In [6]:
colors = {
    'title_background': '#111111',
    'area_background': '#19202A',
    'text': '#FEFEFE',
    'Tested' :'#AEAEAE',
    'Confirmed': '#F4B000',
    'Deaths' :'#E55465',
    'Rate': '#DD1E34'
}

color_scale = [
        "#fadc8f",
        "#f9d67a",
        "#f8d066",
        "#f8c952",
        "#f7c33d",
        "#f6bd29",
        "#f5b614",
        "#F4B000",
        "#eaa900",
        "#e0a200",
        "#dc9e00",
    ]

In [7]:
CONTINENT_LABELS = df['continent'].unique()

In [ ]:
external_stylesheets = [dbc.themes.SLATE,
                        "https://use.fontawesome.com/releases/v5.9.0/css/all.css",
                        ]

# Initialize the app
app = dash.Dash(__name__, 
                external_stylesheets=external_stylesheets
               )

# Define app layout
app.layout = html.Div(style={'backgroundColor': colors['title_background']}, children=[
    html.H1(
        children='COVID 19',
        style={
            'textAlign': 'center',
            'color': colors['text'],
            'padding-left': 14,
        }
    ),
    
    dbc.Row(style={'backgroundColor': colors['area_background']}, children=[
        dbc.Col(dcc.Dropdown(
                    id="continents-dropdown",
                    options=[{'label': i, 'value': i} for i in CONTINENT_LABELS],
                    value="World",
                    clearable=False,
                    style={'border': 0,
                           'padding-left': 7,
                           'font-size': 'large',
                           'color': 'black',
                           'backgroundColor': 'white'
                          }
                    ),
                width=2,
               ),
        
        dbc.Col(dbc.Row(
                        id='daily-stats',
                        style={'backgroundColor': colors['area_background']}
                           ), 
                ),
        ],
    ),        

    dbc.Row(style={'backgroundColor': colors['area_background']}, children=[
        dbc.Col(width=2),
        dbc.Col(dcc.Graph(
                    id='cases_map',
                    style={'color': '#19202A',
                           'border': 0},
                    ),
                width=8,
               ),
        
        dbc.Col(width=2),
    ])
])


@app.callback(
    Output('cases_map', 'figure'),
    Input('continents-dropdown', 'value'))
def update_graph(continent):
    
    if continent != 'World':
        dff = df[df['continent'] == continent]
    else:
        dff = df
    
    scope = {'Europe': 'europe',
             'North America': 'north america',
             'South America': 'south america',
             'Africa': 'africa',
             'Asia': 'asia',
             'Oceania': 'world',
             'World': 'world'
            }
    
    fig = px.scatter_geo(data_frame=dff[dff['date']==dff['date'].max()], 
                     locations="iso_code",
                     size="scaled",
                     color='total_cases',
                     hover_name="location",
                     hover_data=["total_cases", "total_deaths", "location", "continent"],
                     color_continuous_scale=color_scale,
                     basemap_visible=True,
                     template='plotly_dark',
                     scope=scope[continent])

    fig.update_layout(
        margin={"r": 0, "t": 0, "l": 0, "b": 0},
        coloraxis_showscale=False,
        paper_bgcolor= colors['area_background'],
        plot_bgcolor = colors['area_background'],
    )

    return fig

@app.callback(
    Output('daily-stats', 'children'),
    Input('continents-dropdown', 'value')
)
def update_cards(continent):
    
    dff = df[df['date']==df['date'].max()].groupby(['continent'], as_index=False).sum()
    
    stats = {
        "Tested": dff[(dff['continent']==continent)]['new_tests'].item(),
        "Confirmed": [dff[dff['continent']==continent]['total_cases'].item(), dff[dff['continent']==continent]['new_cases'].item()],
        "Deaths": [dff[dff['continent']==continent]['total_deaths'].item(), dff[dff['continent']==continent]['new_deaths'].item()],
        "Rate": dff[dff['continent']==continent]['reproduction_rate'].item(),
    }
    
    cards = []
    
    for key, value in stats.items():
        if (key == "Tested") or (key == 'Rate'):
            card = dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.P(
                                    f"+ {int(value):} new",
                                    style={
                                        'textAlign': 'center',
                                        'color': colors['area_background']
                                    }
                                ),
                                html.H2(
                                    f"{value:}",
                                    style={
                                        'textAlign': 'center',
                                        'backgroundColor': colors['area_background'],
                                        'color': colors[key] 
                                    }
                                ),
                                html.P(f"{key}",
                                       style={
                                        'textAlign': 'center',
                                        'backgroundColor': colors['area_background'],
                                        'color': colors[key]
                                    }),
                            ]
                        ),
                        style={'backgroundColor': colors['area_background'],
                               'border': 0}
                    ),
                    width=3,
                    style={'backgroundColor': colors['area_background']}
                )
        else:
            card = dbc.Col(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.P(
                                    f"+ {int(value[1]):} new",
                                    style={
                                        'textAlign': 'center',
                                        'color': colors[key]
                                    }
                                ),
                                html.H2(
                                    f"{int(value[0]):}",
                                    style={
                                        'textAlign': 'center',
                                        'color': colors[key]
                                    }
                                ),
                                html.P(f"{key}",
                                       style={
                                        'textAlign': 'center',
                                        'color': colors[key]
                                    }),
                            ]
                        ),
                        style={'backgroundColor': colors['area_background'],
                               'border': 0}
                    ),
                    width=3,
                    style={'backgroundColor': colors['area_background']}
                )
        cards.append(card)
    
    return cards

if __name__ == '__main__':
    app.run_server(debug=False)